In [1]:
import torch, jax; print(torch.cuda.is_available()); print(jax.devices())

True
[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]


In [ ]:
PROJ_DIR = '/home/ubuntu/work/CLIP/'
SCRIPT_DIR= PROJ_DIR + "hybrid_clip"
MODEL_DIR= PROJ_DIR + "models/clip-train-v1"
TRAIN_FILE= PROJ_DIR + "data/clip_data_v1/train_v1_700k_aws.json"
VALID_FILE= PROJ_DIR + "data/clip_data_v1/valid_v1_60k_aws.json"
IMAGE_ENCODER= "openai/clip-vit-base-patch32"
TEXT_ENCODER="cardiffnlp/twitter-roberta-base-jun2022"

### Epoch 1-15

In [ ]:
!python3 -W ignore $PROJ_DIR/run_hybrid_clip.py \
    --output_dir $MODEL_DIR \
    --overwrite_output_dir \
    --tokenizer_name=$TEXT_ENCODER \
    --train_file=$TRAIN_FILE \
    --validation_file=$VALID_FILE \
    --do_train --do_eval \
    --num_train_epochs="15" --max_seq_length 24 \
    --per_device_train_batch_size="128" \
    --per_device_eval_batch_size="128" \
    --learning_rate="0.0001" --warmup_ratio 0.1 --weight_decay 0.0 \
    --preprocessing_num_workers 8 \
    --exp_name clip_finetune \
    --eval_when 1 \
    --text_model_name_or_path=$TEXT_ENCODER \
    --vision_model_name_or_path=$IMAGE_ENCODER \
    --freeze_backbones
    #--epoch_offset 44 \
    #--run_from_checkpoint $FROM_CHECKPOINT


## Epoch 15-23

In [ ]:
# Plateaud...
!python3 -W ignore $PROJ_DIR/run_hybrid_clip.py \
    --output_dir $MODEL_DIR \
    --overwrite_output_dir \
    --tokenizer_name=$TEXT_ENCODER \
    --train_file=$TRAIN_FILE \
    --validation_file=$VALID_FILE \
    --do_train --do_eval \
    --num_train_epochs="35" --max_seq_length 24 \
    --per_device_train_batch_size="128" \
    --per_device_eval_batch_size="128" \
    --learning_rate="0.0000015" --warmup_ratio 0.025 --weight_decay 0.0 \
    --preprocessing_num_workers 8 \
    --exp_name clip_finetune \
    --eval_when 1 \
    --text_model_name_or_path=$TEXT_ENCODER \
    --vision_model_name_or_path=$IMAGE_ENCODER \
    --freeze_backbones \
    --epoch_offset 15 \ 
    --run_from_checkpoint $FROM_CHECKPOINT


### Unfreeze Epochs 23 - 

In [ ]:
FROM_CHECKPOINT = MODEL_DIR + "/23"

!python3 -W ignore $PROJ_DIR/run_hybrid_clip.py \
    --output_dir $MODEL_DIR \
    --overwrite_output_dir \
    --tokenizer_name=$TEXT_ENCODER \
    --train_file=$TRAIN_FILE \
    --validation_file=$VALID_FILE \
    --do_train --do_eval \
    --num_train_epochs="30" --max_seq_length 24 \
    --per_device_train_batch_size="128" \
    --per_device_eval_batch_size="128" \
    --learning_rate="0.0001" --warmup_ratio 0.01 --weight_decay 0.0 \
    --preprocessing_num_workers 8 \
    --exp_name clip_finetune \
    --eval_when 1 \
    --text_model_name_or_path=$TEXT_ENCODER \
    --vision_model_name_or_path=$IMAGE_ENCODER \
    --epoch_offset 23 \
    --run_from_checkpoint $FROM_CHECKPOINT